In [ ]:
import numpy as np
import tensorflow as tf

import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import os

CustomData = input("Enter the name of the Dataset : ")

dataset_path = os.listdir(CustomData)

import gradio as gr

import PIL
from tensorflow.keras import layers
from keras.layers.core import Dense
from keras.layers.core import Flatten
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam

import pathlib
data_dir = tf.keras.utils.get_file(CustomData, origin=dataset_path, untar=True)
data_dir = pathlib.Path(data_dir)

print(data_dir)

from PIL import Image

Type1Haks = list(data_dir.glob('Type1Haks/*'))
print(Type1Haks[0])
Image.open(str(Type1Haks[0]))

img_height,img_width=180,180
batch_size=32
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="training",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

val_ds = tf.keras.preprocessing.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="validation",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

class_names = train_ds.class_names
print(class_names)

import matplotlib.pyplot as plt

plt.figure(figsize=(10, 10))
for images, labels in train_ds.take(1):
  for i in range(9):
    ax = plt.subplot(3, 3, i+1)
    plt.imshow(images[i].numpy().astype("uint8"))
    plt.title(class_names[labels[i]])
    plt.axis("off")
    
resnet_model = Sequential()

pretrained_model= tf.keras.applications.ResNet50(include_top=False,
                   input_shape=(180,180,3),
                   pooling='avg',classes=9,
                   weights='imagenet')
for layer in pretrained_model.layers:
        layer.trainable=False

resnet_model.add(pretrained_model)
resnet_model.add(Flatten())
resnet_model.add(Dense(512, activation='relu'))
resnet_model.add(Dense(9, activation='softmax'))

resnet_model.summary()

resnet_model.compile(optimizer=Adam(lr=0.001),loss='sparse_categorical_crossentropy',metrics=['accuracy'])

history = resnet_model.fit(train_ds, validation_data=val_ds, epochs=10)

def predict_image(img):
  img_4d=img.reshape(-1,180,180,3)
  prediction=resnet_model.predict(img_4d)[0]
  return {class_names[i]: float(prediction[i]) for i in range(9)}

image = gr.inputs.Image(shape=(180,180))
label = gr.outputs.Label(num_top_classes=9)

gr.Interface(fn=predict_image, inputs=image, outputs=label,interpretation='default').launch(debug='True')

Enter the name of the Dataset : Images
C:\Users\barna\.keras\datasets\Images
C:\Users\barna\.keras\datasets\Images\Type1Haks\haks1.jpg
Found 2440 files belonging to 9 classes.
Using 1952 files for training.
Found 2440 files belonging to 9 classes.
Using 488 files for validation.
['Type1Haks', 'Type1Pla', 'Type1Range', 'Type1Screw', 'Type1Test', 'Type2Haks', 'Type2Range', 'Type2Screw', 'Type2Test']
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resnet50 (Functional)        (None, 2048)              23587712  
_________________________________________________________________
flatten (Flatten)            (None, 2048)              0         
_________________________________________________________________
dense (Dense)                (None, 512)               1049088   
_________________________________________________________________
dense_1 (Dense)              (None, 9)             

C:\Users\barna\anaconda3\lib\site-packages\gradio\deprecation.py:40: UserWarning: `optional` parameter is deprecated, and it has no effect
  warnings.warn(value)
C:\Users\barna\anaconda3\lib\site-packages\gradio\deprecation.py:40: UserWarning: The 'type' parameter has been deprecated. Use the Number component instead.
  warnings.warn(value)


Running on local URL:  http://127.0.0.1:7860/

To create a public link, set `share=True` in `launch()`.
